In [7]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime

# Load the dataset
data = pd.read_csv("/content/Churn_Modelling.csv")

# Drop irrelevant columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

from sklearn.preprocessing import OneHotEncoder

# One-hot encode 'Geography'
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])

# Convert sparse matrix to dense array
geo_encoder_dense = geo_encoder.toarray()

# Get feature names for one-hot encoded 'Geography'
geo_encoded_df = pd.DataFrame(geo_encoder_dense, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

# Combine one-hot encoded columns with the original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

# Show the processed data
data.head()


# Save the encoders and scaler
with open('/content/label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('/content/onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

# Split the dataset into independent and dependent features
X = data.drop('Exited', axis=1)
y = data['Exited']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

with open('/content/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # HL1 Connected with input layer
    Dense(32, activation='relu'),  # HL2
    Dense(1, activation='sigmoid')  # Output layer
])

# Compile the model
opt = Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100)

# Save the model
model.save('/content/model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8055 - loss: 0.4385 - val_accuracy: 0.8525 - val_loss: 0.3514
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8519 - loss: 0.3589 - val_accuracy: 0.8550 - val_loss: 0.3565
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8526 - loss: 0.3588 - val_accuracy: 0.8605 - val_loss: 0.3444
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8567 - loss: 0.3445 - val_accuracy: 0.8630 - val_loss: 0.3342
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8630 - loss: 0.3274 - val_accuracy: 0.8605 - val_loss: 0.3403
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8614 - loss: 0.3345 - val_accuracy: 0.8550 - val_loss: 0.3466
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8580 - loss: 0.3383 - val_accuracy: 0.8580 - val_loss: 0.3544
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8599 - loss: 0.3378 - val_accu

In [8]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 131.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 9.1 MB/s eta 0:00:00


In [10]:
# Create the app.py file
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import load_model

# Load necessary files for inference
with open('/content/label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('/content/onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('/content/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

# Load the pre-trained model
model = load_model('/content/model.h5')

st.title('Customer Churn Prediction')

# User input
geography = st.selectbox('Geography', onehot_encoder_geo.categories_[0])  # Select geography
gender = st.selectbox('Gender', label_encoder_gender.classes_)  # Select gender
age = st.slider('Age', 18, 92)  # Age slider
balance = st.number_input('Balance')  # Balance input
credit_score = st.number_input('Credit Score')  # Credit score input
estimated_salary = st.number_input('Estimated Salary')  # Salary input
tenure = st.slider('Tenure', 0, 10)  # Tenure slider
num_of_products = st.slider('Number of Products', 1, 4)  # Number of products slider
has_cr_card = st.selectbox('Has Credit Card', [0, 1])  # Credit card checkbox
is_active_member = st.selectbox('Is Active Member', [0, 1])  # Active member checkbox

# Prepare the input data
input_data = pd.DataFrame({
    'CreditScore': [credit_score],
    'Gender': [label_encoder_gender.transform([gender])[0]],  # Transform gender
    'Age': [age],
    'Tenure': [tenure],
    'Balance': [balance],
    'NumOfProducts': [num_of_products],
    'HasCrCard': [has_cr_card],
    'IsActiveMember': [is_active_member],
    'EstimatedSalary': [estimated_salary]
})

# One-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform(pd.DataFrame([[geography]], columns=['Geography'])).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

# Combine one-hot encoded columns with input data
input_data = pd.concat([input_data.reset_index(drop=True), geo_encoded_df], axis=1)

# Scale the input data
input_data_scaled = scaler.transform(input_data)

# Predict churn probability
prediction_proba = model.predict(input_data_scaled)[0][0]  # Getting the probability for class 1 (churn)

# Display result
st.write(f'Churn Probability: {prediction_proba:.2f}')
if prediction_proba > 0.5:
    st.write('The customer is likely to churn.')
else:
    st.write('The customer is not likely to churn.')
""")


In [11]:
from pyngrok import ngrok
import os

# Set the port for Streamlit to run
port = 8501

ngrok.set_auth_token('')

# Open a ngrok tunnel to the streamlit port
public_url = ngrok.connect(port)

# Run Streamlit in the background
os.system("streamlit run app.py &")

# Output the public URL
print(f"Streamlit app is running at: {public_url}")


Streamlit app is running at: NgrokTunnel: "https://a43b-34-125-25-95.ngrok-free.app" -> "http://localhost:8501"
